<a href="https://colab.research.google.com/github/armandochernandez-ai/Curso-python-slava/blob/main/Unidad_3/proceso_admision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `PROYECTO DE ADMISIÓN A FACULTADES.`
# Armando Chávez Hernández.
**Geografía CUCSH.**

## Primera etapa
Generar lista de admitidos facultad por facultad

In [2]:

# Proceso simple para generar listas de admitidos facultad por facultad
from google.colab import drive
import pandas as pd
import sys

instruccion = """Este es un proceso de selección de los mejores aspitantes (puntajes) a ingresar a facultades.
Deberá elegir un cupo para cada facultad, con lo cual se generará una lista de los mejores aspirantes."""

print(instruccion)

# Montar Google Drive si no está montado
drive.mount('/content/drive')

# === 1. Cargar archivo texto a tabla simple ===
#archivo = "/content/drive/MyDrive/Colab Data/applicant_list_7.txt"
#tabla = pd.read_table(archivo)
#tabla


# === 2. Convertir a tabla a dataframe ===
archivo = "/content/drive/MyDrive/Colab Data/applicant_list_7.txt"
#se pueden definir columnas antes
n_columnas = ["Nombre","Apellido","ESP","EMA","EPH","ECH","ECS", "fac_1", "fac_2","fac_3"]
df = pd.read_csv(archivo, sep=' ', names = n_columnas)
df

#O cargar y poner Nombre a columnas
df = "/content/drive/MyDrive/Colab Data/applicant_list_7.txt"
df = pd.read_csv(df, sep=' ', header=None)
#df = pd.DataFrame(df)
df

#Colocar encabezados
df.columns = ["Nombre","Apellido","ESP","EMA","EPH","ECH","ECS", "fac_1", "fac_2","fac_3"]
df

# Crear el esquema para traducir al español
traduccion = {
    'Physics': 'Física',
    'Engineering': 'Ingeniería',
    'Mathematics': 'Matemáticas',
    'Biotech': 'Biotecnología',
    'Chemistry': 'Química'
}

# Aplicar la traduccióna las columnas que lo requiern
df['fac_1'] = df['fac_1'].map(traduccion)
df['fac_2'] = df['fac_2'].map(traduccion)
df['fac_3'] = df['fac_3'].map(traduccion)

# Presentar
df.head()

# Función para buscar columna
def buscar(df, facultad):
  """Búsqueda en data frame en la columna fac_1 para filtar despues."""
  return df[df['fac_1'].str.contains(facultad, case=False, na=False)]
print()
# Pregunta
facultad = str(input("¿Que facultad busca? ")).lower()

try:
    # Búsqueda de resultados en columna
    resultados_busqueda = buscar(df, facultad)

    # Resultados posibles
    if resultados_busqueda.empty:
        # En caso de no encontrar facultad
        raise ValueError(f"'{facultad}'")
    else:
      print(f"Facultad encontrada: {facultad}")
      #Cuando se encuentra

except ValueError as no:
    # Reporte
    print(f"Facultad {no} no existe")
    #sys.exit()

except Exception as e:
    # Otros errores
    print(f"Error inesperado: {e}")
print()
# Vamos a definir el cupo de alumnos
try:
    cupo = int(input(f"¿Cuántos lugares hay para facultad de {facultad}? "))
    if cupo < 0:
        raise ValueError("El número de cupos debe ser mayor que 0.")
except ValueError as e:
    print(f"Error: {e}")
    sys.exit()

print(f"Se ha determinado {cupo} lugares para facultad de {facultad}.")

def calc_promedio(row, facultad):
    """Calcular promedios por facultad."""
    if facultad in ['Física', 'Ingeniería', 'Matemáticas']:
        promedio = row[['ESP', 'EMA', 'EPH']]
    elif facultad in ['Química', 'Biotecnología']:
        promedio = row[['EMA', 'ECH', 'ECS']]
    else:
        return None  # Para caso de facultades no reconocidas

    return promedio.mean()

# Lista de facultades psibles
posibles = list(traduccion.values())

# Iteracion a las facultades psobles
for facultad_nombre in posibles:
  column_name = f'{facultad_nombre}_Promedio'
  df[column_name] = df.apply(calc_promedio, axis=1, facultad=facultad_nombre)

df.head()

# Normalizar resultados
df['fac_1_normalized'] = df['fac_1'].str.lower().str.strip()
df['fac_2_normalized'] = df['fac_2'].str.lower().str.strip()
df['fac_3_normalized'] = df['fac_3'].str.lower().str.strip()
df.head()

# Convertir textos de nombres de facultades a minusculas
facultad_lower = facultad.lower()

# Filtar para seleccionar de columna fac_1_normalizado igual a nombre minusculas
candidatos_fac1 = df[df['fac_1_normalized'] == facultad_lower].copy()

# Calcula promedio para cada candidadto de fac_1
candidatos_fac1['Promedio'] = candidatos_fac1.apply(calc_promedio, axis=1, facultad=facultad.capitalize())

# Ordena de forma ascendente por columna Promedio
candidatos_fac1 = candidatos_fac1.sort_values(by='Promedio', ascending=False)

# Selecciona los mejores de acuerdo al cupo
seleccion_fac1 = candidatos_fac1.head(cupo)
print()
# Lista de fac_1
print(f"Lista admitidos a {facultad.capitalize()} como primera opción:")
print()
display(seleccion_fac1[['Nombre', 'Apellido', 'fac_1', 'Promedio']])
print()

#Definir cupo remanente
cupo_disponible = cupo - len(seleccion_fac1)
print(f"Cupo disponible para la facultad de {facultad.capitalize()} {cupo_disponible} lista de candidatos:")

print()
#Definir candidatos a ocupar espacios disponibles de la segunda y tercera opción
candidatos = df[~df.index.isin(seleccion_fac1.index)].copy()
#Desplegar lista de candidatos
display(candidatos[['Nombre', 'Apellido', 'fac_2', 'fac_3']])

print()
seleccion_otras = []

for index, row in candidatos.iterrows():
    candidatos_data = row.to_dict()
    candidatos_data['Promedio'] = None # Inicializar promedio

    if row['fac_2_normalized'] == facultad_lower:
        candidatos_data['Promedio'] = calc_promedio(row, facultad.capitalize())
        seleccion_otras.append(candidatos_data)
    elif row['fac_3_normalized'] == facultad_lower:
        candidatos_data['Promedio'] = calc_promedio(row, facultad.capitalize())
        seleccion_otras.append(candidatos_data)

# Convertir a DataFrame
seleccion_otras_df = pd.DataFrame(seleccion_otras)
print(f"Encabezados de lista convertida a DataFrame:")
print()
display(seleccion_otras_df.head())
print()
#Selección final
seleccion_otras_df = seleccion_otras_df.sort_values(by='Promedio', ascending=False)
candidatos_otras = min(cupo_disponible, len(seleccion_otras_df))
seleccion_final_otras = seleccion_otras_df.head(candidatos_otras)
estudiantes_admitidos = pd.concat([seleccion_fac1, seleccion_final_otras], ignore_index=True)
print()
print(f"Lista final de estudiantes admitidos {facultad.capitalize()}:")
print()
lista_final = estudiantes_admitidos.sort_values(by='Promedio', ascending=False)
display(lista_final[['Nombre', 'Apellido', 'fac_1', 'Promedio']])

Este es un proceso de selección de los mejores aspitantes (puntajes) a ingresar a facultades.
Deberá elegir un cupo para cada facultad, con lo cual se generará una lista de los mejores aspirantes.
Mounted at /content/drive

¿Que facultad busca? química
Facultad encontrada: química

¿Cuántos lugares hay para facultad de química? 20
Se ha determinado 20 lugares para facultad de química.

Lista admitidos a Química como primera opción:



,Nombre,Apellido,fac_1,Promedio
47,Kennedy,Barrett,Química,91.333333
81,Ayeshia,Jackman,Química,86.666667
62,Estephanie,Phelps,Química,82.000000
2,Uzma,Naysmythe,Química,81.666667
14,Franki,Dinnis,Química,81.666667
50,Crescentia,Dow,Química,75.000000
9,Delta,Fanny,Química,73.333333
75,Brittania,Denny,Química,72.333333
79,Stacey,Revill,Química,71.000000
74,Pearl,Pullins,Química,66.000000



Cupo disponible para la facultad de Química 9 lista de candidatos:



,Nombre,Apellido,fac_2,fac_3
0,Jermine,Brunton,Ingeniería,Matemáticas
1,Justo,Mirfin,Biotecnología,Química
3,Koury,Wingo,Biotecnología,Matemáticas
4,Kentrell,Hillhouse,Ingeniería,Biotecnología
5,Trica,Macalpine,Matemáticas,Biotecnología
...,...,...,...,...
93,Tamkia,Fish,Ingeniería,Matemáticas
94,Deniz,Blanchard,Biotecnología,Química
95,Mira,Riley,Biotecnología,Matemáticas
96,Loura,Macansh,Física,Matemáticas



Encabezados de lista convertida a DataFrame:



,Nombre,Apellido,ESP,EMA,EPH,ECH,ECS,fac_1,fac_2,fac_3,Física_Promedio,Ingeniería_Promedio,Matemáticas_Promedio,Biotecnología_Promedio,Química_Promedio,fac_1_normalized,fac_2_normalized,fac_3_normalized,Promedio
0,Justo,Mirfin,71,77,61,60,41,Ingeniería,Biotecnología,Química,69.666667,69.666667,69.666667,59.333333,59.333333,ingeniería,biotecnología,química,59.333333
1,Laney,Braithwaite,90,90,90,72,94,Física,Química,Matemáticas,90.000000,90.000000,90.000000,85.333333,85.333333,física,química,matemáticas,85.333333
2,Spring,Burridge,71,84,98,71,50,Matemáticas,Química,Ingeniería,84.333333,84.333333,84.333333,68.333333,68.333333,matemáticas,química,ingeniería,68.333333
3,Elen,Ashbury,54,76,88,62,60,Matemáticas,Química,Biotecnología,72.666667,72.666667,72.666667,66.000000,66.000000,matemáticas,química,biotecnología,66.000000
4,Artavious,Fay,71,77,61,60,61,Ingeniería,Biotecnología,Química,69.666667,69.666667,69.666667,66.000000,66.000000,ingeniería,biotecnología,química,66.000000




Lista final de estudiantes admitidos Química:



,Nombre,Apellido,fac_1,Promedio
0,Kennedy,Barrett,Química,91.333333
1,Ayeshia,Jackman,Química,86.666667
11,Dashanna,Herron,Física,86.666667
12,Laney,Braithwaite,Física,85.333333
13,Blia,Sagar,Física,82.333333
14,Genee,Mccrae,Biotecnología,82.333333
2,Estephanie,Phelps,Química,82.000000
3,Uzma,Naysmythe,Química,81.666667
15,Madiha,Milligan,Física,81.666667
4,Franki,Dinnis,Química,81.666667


## Segunda etapa
Generar lista de admitidos para todas las facultades con puntajes.
Se consideran todas las opciones por orden e importancia.

In [10]:
# Lista para almacenar estudiantes admitidos y no admitidos
estudiantes_admitidos_total = pd.DataFrame()
estudiantes_no_admitidos_total = pd.DataFrame()

# Iterar sobre cada facultad posible
for facultad_nombre in posibles:
    print(f"\n--- Procesando facultad: {facultad_nombre} ---")
    print()
    # Preguntar por el cupo para la facultad actual
    while True:
        try:
            cupo = int(input(f"¿Cuántos cupos hay para {facultad_nombre}? "))
            if cupo < 0:
                raise ValueError("El número de cupos debe ser mayor que 0.")
            break # Salir del bucle si la entrada es válida
        except ValueError as e:
            print(f"Error: {e}")

    print()
    print(f"Se han definido {cupo} cupos para {facultad_nombre}.")

    # Convertir nombre de facultad a minusculas para comparación
    facultad_lower = facultad_nombre.lower()

    # Filtrar candidatos con fac_1 igual a la facultad actual
    candidatos_fac1 = df[df['fac_1_normalized'] == facultad_lower].copy()

    # Calcular promedio para cada candidato de fac_1
    candidatos_fac1['Promedio'] = candidatos_fac1.apply(calc_promedio, axis=1, facultad=facultad_nombre)

    # Ordenar de forma descendente por columna Promedio
    candidatos_fac1 = candidatos_fac1.sort_values(by='Promedio', ascending=False)

    # Seleccionar los mejores de acuerdo al cupo (primera opción)
    seleccion_fac1 = candidatos_fac1.head(cupo)

    print()
    # Lista de admitidos de primera opción
    print(f"Lista admitidos a {facultad_nombre} como primera opción:")
    print()
    display(seleccion_fac1[['Nombre', 'Apellido', 'fac_1', 'Promedio']])

    # Definir cupo remanente
    cupo_disponible = cupo - len(seleccion_fac1)
    print()
    print(f"Cupo disponible para {facultad_nombre}: {cupo_disponible}")

    print()
    # Si hay cupo disponible, considerar segunda y tercera opción
    seleccion_otras = []
    if cupo_disponible > 0:
        print(f"Considerando candidatos de segunda y tercera opción para {facultad_nombre}...")
        # Candidatos que no fueron admitidos en la primera opción para esta facultad
        candidatos_remanentes = df[~df.index.isin(seleccion_fac1.index)].copy()
        print()

        for index, row in candidatos_remanentes.iterrows():
            candidatos_data = row.to_dict()
            candidatos_data['Promedio'] = None # Inicializar promedio

            if row['fac_2_normalized'] == facultad_lower:
                candidatos_data['Promedio'] = calc_promedio(row, facultad_nombre)
                seleccion_otras.append(candidatos_data)
            elif row['fac_3_normalized'] == facultad_lower:
                candidatos_data['Promedio'] = calc_promedio(row, facultad_nombre)
                seleccion_otras.append(candidatos_data)

        # Convertir a DataFrame
        seleccion_otras_df = pd.DataFrame(seleccion_otras)

        print()
        # Si hay candidatos de otras opciones, ordenarlos y seleccionar
        if not seleccion_otras_df.empty:
            seleccion_otras_df = seleccion_otras_df.sort_values(by='Promedio', ascending=False)
            candidatos_otras_a_seleccionar = min(cupo_disponible, len(seleccion_otras_df))
            seleccion_final_otras = seleccion_otras_df.head(candidatos_otras_a_seleccionar)
            print(f"Admitidos de segunda y tercera opción para {facultad_nombre}:")
            display(seleccion_final_otras[['Nombre', 'Apellido', 'fac_2', 'fac_3', 'Promedio']])
            print()
        else:
            seleccion_final_otras = pd.DataFrame() # DataFrame vacío si no hay candidatos de otras opciones

    else:
        seleccion_final_otras = pd.DataFrame() # DataFrame vacío si no hay cupo disponible

    # Concatenar admitidos de primera opción y de otras opciones
    estudiantes_admitidos_facultad = pd.concat([seleccion_fac1, seleccion_final_otras], ignore_index=True)

    # Agregar columna de facultad admitida para los de segunda y tercera opción
    estudiantes_admitidos_facultad['Facultad_Admitida'] = facultad_nombre

    # Unir a la lista total de admitidos
    estudiantes_admitidos_total = pd.concat([estudiantes_admitidos_total, estudiantes_admitidos_facultad], ignore_index=True)

    # Identificar estudiantes no admitidos para esta facultad (los que no están en la lista de admitidos_facultad)
    # Consideramos a todos los que tenían esta facultad como opción 1, 2 o 3 pero no fueron admitidos
    candidatos_facultad_opciones = df[
        (df['fac_1_normalized'] == facultad_lower) |
        (df['fac_2_normalized'] == facultad_lower) |
        (df['fac_3_normalized'] == facultad_lower)
    ].copy()

    # Estudiantes no admitidos en esta facultad son aquellos en candidatos_facultad_opciones que NO están en estudiantes_admitidos_facultad
    estudiantes_no_admitidos_facultad = candidatos_facultad_opciones[
        ~candidatos_facultad_opciones['Nombre'].isin(estudiantes_admitidos_facultad['Nombre']) |
        ~candidatos_facultad_opciones['Apellido'].isin(estudiantes_admitidos_facultad['Apellido'])
    ].copy()


    # Calcular el promedio relevante para los no admitidos si no está ya calculado
    estudiantes_no_admitidos_facultad['Promedio'] = estudiantes_no_admitidos_facultad.apply(
        lambda row: calc_promedio(row, facultad_nombre) if pd.isna(row.get('Promedio')) else row['Promedio'], axis=1
    )

     # Unir a la lista total de no admitidos (evitando duplicados si un estudiante no fue admitido en varias facultades)
    estudiantes_no_admitidos_total = pd.concat([estudiantes_no_admitidos_total, estudiantes_no_admitidos_facultad], ignore_index=True).drop_duplicates(subset=['Nombre', 'Apellido'])


# Presentar lista final de admitidos (ordenada por promedio general, si es posible, o por facultad)
print("\n--- Lista Final de Estudiantes Admitidos en Todas las Facultades ---")
# Para una presentación más limpia, seleccionamos solo las columnas relevantes y renombramos 'Facultad_Admitida'
lista_final_admitidos_presentacion = estudiantes_admitidos_total[['Nombre', 'Apellido', 'Facultad_Admitida', 'Promedio']].copy()
lista_final_admitidos_presentacion = lista_final_admitidos_presentacion.sort_values(by= ['Facultad_Admitida','Promedio'], ascending=False)
print()
display(lista_final_admitidos_presentacion)


# Presentar lista final de no admitidos
print("\n--- Lista Final de Estudiantes No Admitidos ---")
# Para los no admitidos, mostramos el promedio de la facultad que era su primera opción (si aplica) o el promedio general si se calculó
estudiantes_no_admitidos_total_presentacion = estudiantes_no_admitidos_total[['Nombre', 'Apellido', 'Promedio']].copy()
estudiantes_no_admitidos_total_presentacion = estudiantes_no_admitidos_total_presentacion.sort_values(by='Promedio', ascending=False)
print()
display(estudiantes_no_admitidos_total_presentacion)


--- Procesando facultad: Física ---

¿Cuántos cupos hay para Física? 18

Se han definido 18 cupos para Física.

Lista admitidos a Física como primera opción:



,Nombre,Apellido,fac_1,Promedio
7,Laney,Braithwaite,Física,90.000000
54,Cressie,Gillespie,Física,86.333333
18,Cornellius,Turney,Física,85.333333
6,Sang,Muldoon,Física,85.333333
42,Martha,Hatchard,Física,85.333333
30,Jathan,Birley,Física,85.333333
19,Blia,Sagar,Física,83.666667
36,Derick,Whatley,Física,81.666667
84,Jermain,Stobbs,Física,81.666667
0,Jermine,Brunton,Física,79.333333



Cupo disponible para Física: 0


--- Procesando facultad: Ingeniería ---

¿Cuántos cupos hay para Ingeniería? 20

Se han definido 20 cupos para Ingeniería.

Lista admitidos a Ingeniería como primera opción:



,Nombre,Apellido,fac_1,Promedio
87,Melena,Hearn,Ingeniería,86.000000
39,Jamarl,Delap,Ingeniería,86.000000
21,Linda,Risley,Ingeniería,84.333333
44,Luna,Pheobe,Ingeniería,84.333333
53,Nicolasa,Sumpter,Ingeniería,84.333333
5,Trica,Macalpine,Ingeniería,83.666667
89,Amira,Giddings,Ingeniería,83.666667
41,Kaylie,Lanthis,Ingeniería,83.666667
78,Ronel,Cowan,Ingeniería,83.666667
17,Shantale,Tompkins,Ingeniería,83.666667



Cupo disponible para Ingeniería: 0


--- Procesando facultad: Matemáticas ---

¿Cuántos cupos hay para Matemáticas? 25

Se han definido 25 cupos para Matemáticas.

Lista admitidos a Matemáticas como primera opción:



...

,Nombre,Apellido,fac_2,fac_3,Promedio
12,Dashanna,Herron,Química,Matemáticas,86.666667
1,Laney,Braithwaite,Química,Matemáticas,85.333333
5,Blia,Sagar,Química,Biotecnología,82.333333
14,Genee,Mccrae,Química,Física,82.333333
9,Madiha,Milligan,Ingeniería,Química,81.666667
10,Tawnia,Alcock,Química,Física,81.000000
27,Mir,Ashley,Física,Química,79.333333
11,Quinisha,Clarkson,Química,Ingeniería,78.333333
16,Randon,Bradhust,Ingeniería,Química,76.666667
18,Kennon,Inverarity,Ingeniería,Química,75.666667




--- Lista Final de Estudiantes Admitidos en Todas las Facultades ---



,Nombre,Apellido,Facultad_Admitida,Promedio
78,Kennedy,Barrett,Química,91.333333
79,Ayeshia,Jackman,Química,86.666667
89,Dashanna,Herron,Química,86.666667
90,Laney,Braithwaite,Química,85.333333
91,Blia,Sagar,Química,82.333333
...,...,...,...,...
63,Genee,Mccrae,Biotecnología,82.333333
64,Tawnia,Alcock,Biotecnología,81.000000
65,Randon,Bradhust,Biotecnología,76.666667
66,Narissa,Worthington,Biotecnología,75.000000



--- Lista Final de Estudiantes No Admitidos ---



,Nombre,Apellido,Promedio
8,Crescentia,Dow,88.333333
19,Stacey,Revill,86.666667
55,Cressie,Gillespie,86.333333
77,Jamarl,Delap,86.000000
7,Mehul,Bull,86.000000
...,...,...,...
51,Savvas,Hjellstrom,60.333333
92,Dashawn,Bosley,60.000000
45,Marygrace,Wheelton,59.666667
93,Esther,Bratby,56.000000
